# Notebook TESTING lazyloading of AMPS domain 02 RAW output 

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os

import gcsfs
from tqdm import tqdm
import fsspec

xr.set_options(display_style="html");
# xr.show_versions()

In [18]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} wrf-python netCDF4

import wrf
from netCDF4 import Dataset

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



# Dask Startup

In [3]:
# xr.show_versions()

In [3]:
## A chunk of code to start dask cluster
from dask.distributed import Client, progress
from dask_gateway import Gateway
gateway = Gateway()

In [4]:
if gateway.list_clusters():
    cluster_name = gateway.list_clusters()[0].name
    cluster = gateway.connect(cluster_name)
    print(f"\nConnecting to pre-existing cluster with {len(cluster.scheduler_info['workers'])} workers.")
else:
    cluster = gateway.new_cluster() 
    cluster.adapt(minimum=4, maximum=12)
    print("\nStarting up and connecting to new cluster.")

cluster
# cluster.scheduler_info


Starting up and connecting to new cluster.


In [5]:
client = cluster.get_client()
client

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.1439336e036b48beb7470be39853f237 Dashboard: /services/dask-gateway/clusters/prod.1439336e036b48beb7470be39853f237/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


### setup runtime parameters

In [6]:
gcsdir = 'gs://ldeo-glaciology'
ampsdir = 'AMPS'
amps_ver = 'WRF_24'
domain = 'domain_02'
filepattern = 'wrfout_d02_20191231*'


In [7]:
# pattern = 'gs://ldeo-glaciology/AMPS/WRF_24/domain_03/wrf-20161225*'
pattern = os.path.join(gcsdir, ampsdir, amps_ver, domain, filepattern)
print(pattern)

gs://ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_20191231*


### list the netcdf files

In [8]:
fs = gcsfs.GCSFileSystem(project='ldeo-glaciology', mode='ab', cache_timeout = 0)

In [9]:
NCs = fs.glob(pattern)
print(f"Total of {len(NCs)} wrf files.\n")
print(NCs[0])

Total of 8 wrf files.

ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123100_f003.nc


In [10]:
NCs_urls = ['https://storage.googleapis.com/' + x + '#mode=bytes' for x in NCs]
print(NCs_urls)

['https://storage.googleapis.com/ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123100_f003.nc#mode=bytes', 'https://storage.googleapis.com/ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123100_f006.nc#mode=bytes', 'https://storage.googleapis.com/ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123100_f009.nc#mode=bytes', 'https://storage.googleapis.com/ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123100_f012.nc#mode=bytes', 'https://storage.googleapis.com/ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123112_f003.nc#mode=bytes', 'https://storage.googleapis.com/ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123112_f006.nc#mode=bytes', 'https://storage.googleapis.com/ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123112_f009.nc#mode=bytes', 'https://storage.googleapis.com/ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123112_f012.nc#mode=bytes']


## Read in netCDF 

test using xarray AND wrf-python

https://wrf-python.readthedocs.io/en/latest/faq.html#can-i-use-xarray-dataset-as-an-input-to-the-wrf-python-functions


In [28]:
NCs_urls = ['gs://' + x + '#mode=bytes' for x in NCs]
print(NCs_urls[0])


openfile = fs.open(NCs_urls[0], mode='rb') 
ds = xr.open_dataset(openfile, engine='h5netcdf',chunks={'south_north': -1, 
                                                               'west_east': -1,
                                                               'Time': -1})
for i in tqdm(range(1, 8)):
    openfile = fs.open(NCs_urls[i], mode='rb') 
    temp = xr.open_dataset(openfile, engine='h5netcdf',chunks={'south_north': -1, 
                                                               'west_east': -1,
                                                               'Time': -1})
    ds = xr.concat([ds,temp],'Time')



gs://ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123100_f003.nc#mode=bytes


100%|██████████| 7/7 [00:06<00:00,  1.05it/s]


In [21]:
# ds = ds.chunk({'Time': 1})

In [ ]:
ds._file_obj.

In [ ]:
objs = [file_obj_.ds for file_obj_ in ds._file_obj.file_objs]

In [ ]:
wrf.getvar(objs, varname, timeidx=0)

In [ ]:
slp = wrf.getvar(ds._file_obj.ds, "slp")

## wrf-python

### use raw wrf-python functions to do diagnostics (hopefully using Dask Delayed...)

In [16]:
nx = ds.dims['west_east']
ny = ds.dims['south_north']
dt, dx, dy = ds.DT, ds.DX, ds.DY
cen_lat, cen_lon = ds.CEN_LAT, ds.CEN_LON
truelat1, truelat2, STAND_LON = ds.TRUELAT1, ds.TRUELAT2, ds.STAND_LON
pole_lat, pole_lon = ds.POLE_LAT, ds.POLE_LON

In [29]:
cone = 1 # ???
uv   = wrf.uvmet(ds.U10, ds.V10, 
                 ds.XLONG.isel(Time=1), ds.XLAT.isel(Time=1), 
                 cen_lon, cone, meta=True, units='m s-1')

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [17]:
uv

<xarray.DataArray 'uvmet' (u_v: 2, Time: 8, south_north: 786, west_east: 834)>
array([[[[-1.17902048e-01, -1.29071772e-01, -1.26908094e-01, ...,
           3.02273303e-01,  3.63769323e-01,  4.94509220e-01],
         [-9.62983817e-02, -8.90485793e-02, -8.12111199e-02, ...,
           4.74086016e-01,  6.01928771e-01,  7.72971272e-01],
         [-7.34016821e-02, -3.11118122e-02, -7.16429530e-03, ...,
           6.91603482e-01,  8.93858254e-01,  1.07994342e+00],
         ...,
         [-4.13692856e+00, -4.07623053e+00, -4.02439642e+00, ...,
           7.51450729e+00,  7.45834875e+00,  7.44684601e+00],
         [-4.22153568e+00, -4.18166971e+00, -4.11433077e+00, ...,
           7.69830656e+00,  7.65977907e+00,  7.64349413e+00],
         [-4.27691317e+00, -4.23520756e+00, -4.18657446e+00, ...,
           7.90241957e+00,  7.86289597e+00,  7.82877827e+00]],

        [[-2.05131626e+00, -2.00378180e+00, -1.99072206e+00, ...,
           3.04277015e+00,  3.05266666e+00,  3.02241611e+00],
         [-1.95844960e+00, -1.90599275e+00, -1.90325761e+00, ...,
           3.00414252e+00,  3.03479314e+00,  3.08718181e+00],
         [-1.89123082e+00, -1.82008088e+00, -1.81646633e+00, ...,
           3.04988694e+00,  3.07800293e+00,  3.19781756e+00],
...
          -1.43016899e+00, -1.39810812e+00, -1.37702525e+00],
         [-9.51283550e+00, -9.60309887e+00, -9.69522858e+00, ...,
          -1.47083783e+00, -1.42704380e+00, -1.39381874e+00],
         [-9.60644722e+00, -9.72441292e+00, -9.80776405e+00, ...,
          -1.50060940e+00, -1.46808839e+00, -1.41976523e+00]],

        [[-4.30219793e+00, -4.32233238e+00, -4.36071682e+00, ...,
           8.11095428e+00,  8.14750576e+00,  8.19003963e+00],
         [-4.33524418e+00, -4.32566118e+00, -4.35848284e+00, ...,
           8.00306416e+00,  8.04930496e+00,  8.11359501e+00],
         [-4.35148621e+00, -4.35094261e+00, -4.38984156e+00, ...,
           7.90820885e+00,  7.95193624e+00,  8.02473068e+00],
         ...,
         [-1.11369743e+01, -1.10009136e+01, -1.08402071e+01, ...,
           3.83987427e-01,  3.98094416e-01,  4.19225544e-01],
         [-1.10452003e+01, -1.09039879e+01, -1.06951942e+01, ...,
           3.98455769e-01,  3.89869303e-01,  3.92606646e-01],
         [-1.09909773e+01, -1.08267183e+01, -1.05859604e+01, ...,
           3.59676421e-01,  3.46100777e-01,  3.37106556e-01]]]],
      dtype=float32)
Coordinates:
  * u_v      (u_v) <U1 'u' 'v'
Dimensions without coordinates: Time, south_north, west_east
Attributes:
    units:        m s-1
    description:  earth rotated u,v

### use wrf-python `getvar` to read in diagnostic variables
but they aren't dask, e.g. no distributed

In [ ]:
nc = Dataset(NCs_urls[0])

In [ ]:
t2 = wrf.getvar(nc, 'T2', timeidx=wrf.ALL_TIMES)
# t2 = wrf.getvar(nc, 'T2', timeidx=2) # extract 3rd time instance (t=2) - slow....
t2

In [ ]:
plt.pcolor(wrf.to_np(t2))

In [ ]:
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
import cartopy.feature as cfe
import cartopy.crs as crs

# select one time instance if you have retrieved ALL_TIMES
# t2 = t2.isel(Time=1)

# Get the latitude and longitude points (use original data, rather than any processed data)
lats, lons = wrf.latlon_coords(t2)

# Get the cartopy mapping object (use original data, rather than any processed data)
cart_proj = wrf.get_cartopy(t2)

# Create a figure
fig = plt.figure(figsize=(12,9))
# Set the GeoAxes to the projection used by WRF
ax = plt.axes(projection=cart_proj)

# Add coastlines
ax.coastlines('50m', linewidth=0.8)
ax.add_feature(cfe.NaturalEarthFeature('physical', 'antarctic_ice_shelves_lines', 
                                       '50m', linewidth=1.0, edgecolor='k', facecolor='none') )

# Plot contours
plt.contourf(wrf.to_np(lons), wrf.to_np(lats), wrf.to_np(t2), 30, 
                transform=crs.PlateCarree(), cmap=get_cmap("Spectral"))

# Add a color bar
cbar = plt.colorbar(ax=ax, shrink=.62)
cbar.set_label(t2.units)

# Set the map limits.  Not really necessary, but used for demonstration.
ax.set_xlim(np.array(wrf.cartopy_xlim(t2))/2)
ax.set_ylim(np.array(wrf.cartopy_ylim(t2))/2)

# Add the gridlines
ax.gridlines(color="black", linestyle="dotted")

plt.title(t2.description+'\n'+str(t2.Time.values))

print('')

In [ ]:
## THIS NEEDS cartopy 0.18 but can't install it for some reason...

import shapely.geometry as sgeom
import cartopy.crs as ccrs


box = sgeom.box(minx=35, maxx=175, miny=-80, maxy=-88.5)
x0, y0, x1, y1 = box.bounds
myproj = ccrs.SouthPolarStereo(central_longitude=180)
# myproj = ccrs.Orthographic()

plt.figure(figsize=(4, 5), facecolor='white', dpi=144)
ax = plt.axes(projection=myproj)
ax.set_extent([x0, x1, y0, y1], ccrs.PlateCarree())  
ax.coastlines('50m')
# ax.stock_img()

# pcm1 = ax.pcolormesh(da.west_east, ds.south_north, da.mean(dim='Time'),\
#                     vmin=268, vmax=275,\
#                     transform=ccrs.PlateCarree())#, cmap=plt.get_cmap("BuPu")
## Alternative
kwargs = dict(ax=ax,
              x='west_east', y='south_north',
              transform=ccrs.PlateCarree())#, vmin=270, vmax=275)
pcm1 = t2.plot.contourf(levels=31, robust=True,  **kwargs)


# # c = plt.colorbar(orientation='vertical', shrink=0.4, pad=0.10)
# cb1 = plt.colorbar(pcm1, ax=ax, label=r'Surface Heat Flux $[W {m^{-2}}]$', \
#                    orientation='horizontal', extend='both', \
#                   shrink=0.9, pad=0.01)
# ax.plot(ds.lon[si, sj], ds.lat[si, sj], 'ko', markersize=3, transform=ccrs.PlateCarree())
# ax.plot(ds.lon[ai, aj], ds.lat[ai, aj], 'ko', markersize=3, transform=ccrs.PlateCarree())
# # ax.plot(LON[ssj, ssi], LAT[ssj, ssi], 'ko', markersize=2, transform=ccrs.PlateCarree())

ax.gridlines(color="black", linestyle="dotted")
# plt.suptitle('LH')
# plt.tight_layout()
# plt.savefig('figs/mapplot_TNB_Fsfc_mean.png')
plt.show()

In [ ]:
# Use SLP for the example variable
slp = wrf.getvar(nc, "slp")

# Get the cartopy mapping object
cart_proj = wrf.get_cartopy(slp)

print (cart_proj)

# Get the latitude and longitude coordinate.  This is usually needed for plotting.
lats, lons = wrf.latlon_coords(slp)

# Get the geobounds for the SLP variable
bounds = wrf.geo_bounds(slp)

print (bounds)

# Get the geographic boundaries for a subset of the domain
slp_subset = slp[150:250, 150:250]
slp_subset_bounds = wrf.geo_bounds(slp_subset)

print (slp_subset_bounds)

### wrf-python on multiple WRF files

In [ ]:
# ncfile = nc.MFDataset(NCs_urls)

# T2 = wrf.combine_files(nc, timeidx="ALL_TIMES")#, method= "join", meta="False")

In [16]:
# # Creating a simple test list with three timesteps
# wrflist = [Dataset(NCs_urls[0]),
#            Dataset(NCs_urls[1]),
#            Dataset(NCs_urls[2]),
#            Dataset(NCs_urls[3])]

wrflist = [Dataset(x) for x in NCs_urls]

OSError: [Errno -36] NetCDF: Invalid argument: b'gs://ldeo-glaciology/AMPS/WRF_24/domain_02/wrfout_d02_2019123100_f003.nc#mode=bytes'

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.8/ssl.py", line 1309, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1124)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/p

In [ ]:
# Extract the 'P' variable for all times
T2_join = wrf.getvar(wrflist, "T2", timeidx=wrf.ALL_TIMES, method='cat')#, method="join", squeeze=False)

In [ ]:
print(T2_join)

In [ ]:
T2_join.mean(dim='Time').plot()

In [ ]:
# T2_join.hvplot.image('west_east', 'south_north',
#                     groupby='Time', rasterize=True, dynamic=True, width=800, height=600,
#                     widget_type='scrubber', widget_location='bottom', cmap='RdBu_r')

### convert to DaskArray

In [ ]:
da = T2_join.chunk(chunks={'Time':1})
da

In [ ]:
# da.XLONG.plot()

In [ ]:
# %%time
da.mean(dim='Time').plot()

In [ ]:
%%time
da.mean().compute()

In [ ]:
ds = da.to_dataset()
ds

### cartopy and shapely plot

In [ ]:
## THIS NEEDS cartopy 0.18 but can't install it for some reason...

import shapely.geometry as sgeom
import cartopy.crs as ccrs


box = sgeom.box(minx=35, maxx=175, miny=-80, maxy=-88.5)
x0, y0, x1, y1 = box.bounds
myproj = ccrs.SouthPolarStereo(central_longitude=180)
# myproj = ccrs.Orthographic()

plt.figure(figsize=(4, 5), facecolor='white', dpi=144)
ax = plt.axes(projection=myproj)
ax.set_extent([x0, x1, y0, y1], ccrs.PlateCarree())  
ax.coastlines('50m')
# ax.stock_img()

# pcm1 = ax.pcolormesh(da.west_east, ds.south_north, da.mean(dim='Time'),\
#                     vmin=268, vmax=275,\
#                     transform=ccrs.PlateCarree())#, cmap=plt.get_cmap("BuPu")
## Alternative
kwargs = dict(ax=ax,
              x='west_east', y='south_north',
              transform=ccrs.PlateCarree())#, vmin=270, vmax=275)
pcm1 = da.mean(dim='Time').plot.contourf(levels=31, robust=True,  **kwargs)


# # c = plt.colorbar(orientation='vertical', shrink=0.4, pad=0.10)
# cb1 = plt.colorbar(pcm1, ax=ax, label=r'Surface Heat Flux $[W {m^{-2}}]$', \
#                    orientation='horizontal', extend='both', \
#                   shrink=0.9, pad=0.01)
# ax.plot(ds.lon[si, sj], ds.lat[si, sj], 'ko', markersize=3, transform=ccrs.PlateCarree())
# ax.plot(ds.lon[ai, aj], ds.lat[ai, aj], 'ko', markersize=3, transform=ccrs.PlateCarree())
# # ax.plot(LON[ssj, ssi], LAT[ssj, ssi], 'ko', markersize=2, transform=ccrs.PlateCarree())

ax.gridlines(color="black", linestyle="dotted")
# plt.suptitle('LH')
# plt.tight_layout()
# plt.savefig('figs/mapplot_TNB_Fsfc_mean.png')
plt.show()

In [ ]:
from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cfeature

plt.figure(figsize=(10,5), dpi= 90)

# here is where you specify what projection you want to use
ax = plt.axes(projection=ccrs.PlateCarree())

# here is here you tell Cartopy that the projection
# of your 'x' and 'y' are geographic (lons and lats)
# and that you want to transform those lats and lons
# into 'x' and 'y' in the projection
plt.contourf(da.west_east, ds.south_north, da.mean(dim='Time'), 60,
             transform=ccrs.PlateCarree());
# kwargs = dict(ax=ax,
#               x='west_east', y='south_north',
#               transform=ccrs.PlateCarree())#, vmin=270, vmax=275)
# pcm1 = da.mean(dim='Time').plot.contourf(levels=31, robust=True,  **kwargs)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--');
ax.coastlines()
ax.add_feature(cfeature.LAND)

plt.colorbar()

## Test write to zarr

In [ ]:
outdir = os.path.join(gcsdir, ampsdir, amps_ver, domain, 'zarr-cf/')
# fs.glob(outdir)
fs.ls(outdir)

In [ ]:
import json

with open('secrets/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)
# gcs = gcsfs.GCSFileSystem(token=token)

amps_mapper = fsspec.get_mapper(outdir + 'test_20161225-cf.zarr', mode='ab',
                            token=token)
# ds.to_zarr(amps_mapper, mode='w');

## Close your cluster, be a good denizen.

In [30]:
cluster.shutdown()